# Basic usage of the GraphRAG-SDK to Create a Knowledge Graph and RAG System

The following example demonstrates the basic usage of this SDK to create a GraphRAG using URLs with auto-detected ontology.

In [ ]:
!pip install graphrag_sdk[litellm]

In [1]:
import os
import logging
from dotenv import load_dotenv
from graphrag_sdk.source import URL
from graphrag_sdk import KnowledgeGraph, Ontology
from graphrag_sdk.models.litellm import LiteModel
from graphrag_sdk.model_config import KnowledgeGraphModelConfig

# Load environment variables
load_dotenv()
logging.disable(logging.CRITICAL)

Configurations

In [ ]:
# Get your Credentails from google website, see https://aistudio.google.com/
os.environ["GEMINI_API_KEY"] = ""

# Get your Credentails from Falkor website, see https://app.falkordb.cloud
falkor_host = ""
falkor_port = None
falkor_username = ""
falkor_password = ""

### Import Source Data from URLs

This example uses rottentomatoes URL files as the source data. We will import these files as `Source` objects.

In [3]:
urls = [
    "https://www.rottentomatoes.com/m/side_by_side_2012",
    "https://www.rottentomatoes.com/m/matrix",
    "https://www.rottentomatoes.com/m/matrix_revolutions",
    "https://www.rottentomatoes.com/m/matrix_reloaded",
    "https://www.rottentomatoes.com/m/speed_1994",
    "https://www.rottentomatoes.com/m/john_wick_chapter_4",
]

sources = [URL(url) for url in urls]

### Ontology from the Sources

Next, we will utilize an LLM to automatically extract ontology from the data. We will also add `boundaries` to the ontology detection process to ensure the desired ontology is accurately identified.

In [4]:
model = LiteModel(model_name="gemini/gemini-2.0-flash-exp")

boundaries = """
    Extract only the most relevant information about all the movies, actors, and directors over the text.
    Avoid creating entities for details that can be expressed as attributes.
"""

ontology = Ontology.from_sources(
    sources=sources,
    boundaries=boundaries,
    model=model,
)

### KG from Sources and Ontology

Create a Knowledge Graph (KG) from the sources.

In [5]:
kg = KnowledgeGraph(
    name="movies",
    model_config=KnowledgeGraphModelConfig.with_model(model),
    ontology=ontology,
    host=falkor_host,
    port=falkor_port,
    username=falkor_username,
    password=falkor_password,
)
kg.process_sources(sources)

### Graph RAG

Utilize the `chat_session` method to start a conversations.

In [6]:
chat = kg.chat_session()

answer = chat.send_message("Who is the director of the movie The Matrix?")
print(f"Q: {answer['question']} \nA: {answer['response']}\n")

answer = chat.send_message("How this director connected to Keanu Reeves?")
print(f"Q: {answer['question']} \nA: {answer['response']}\n")

answer = chat.send_message("Who is the director of the movie Side by Side?")
print(f"Q: {answer['question']} \nA: {answer['response']}\n")

answer = chat.send_message(
    "Order the directors that you mentioned in all of our conversation by lexical order."
)
print(f"Q: {answer['question']} \nA: {answer['response']}\n")

Q: Who is the director of the movie The Matrix? 
A: Lilly Wachowski and Lana Wachowski are the directors of the movie The Matrix.


Q: How this director connected to Keanu Reeves? 
A: Lilly Wachowski and Lana Wachowski directed movies in which Keanu Reeves acted.


Q: Who is the director of the movie Side by Side? 
A: Christopher Kenneally is the director of the movie Side by Side.


Q: Order the directors that you mentioned in all of our conversation by lexical order. 
A: The directors in lexical order are: Christopher Kenneally, Lana Wachowski, and Lilly Wachowski.


